# Lab: Neural Net

(Last update: 16/04/2022)

- Student ID: 
- Student name:

---

You will work directly on this notebook; the word `TODO` indicates the part you need to do.  You can discuss ideas with classmates as well as find information from the internet, book, ... but *this lab must be your*.

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`). Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`). Copy the notebook file (*do not copy the data file*) to this folder, compress and submit it on moodle.

---

**What problem do we want to solve in this lab?**

Given the training data: 
$$\{(\textbf{x}^{(1)}, y^{(1)}), ..., (\textbf{x}^{(N)}, y^{(N)})\}$$
where:

- $\textbf{x}^{(n)} \in \mathbb{R}^{784}$ is an input vector containing pixel values of a $28 \times 28$ grayscale image of some hand-written digit
- $y^{(n)} \in \{0, 1, 2, ..., 9\}$ is the corresponding output indicating which digit

Our task is building a model (in this lab: a Fully-Connected Feed-Forward Neural Net) from this data so that it can take a *new* hand-written digit image (a vector $\in \mathbb{R}^{784}$) as input and predict the output (which digit) *well*.


Are you ready? Let's start!

## Import Python libraries

In [ ]:
# You will use Numpy as main lib in this lab
# So, you should use Numpy operations on Numpy arrays, avoid using loops;
# otherwise, the code will run slow
import numpy as np 

# Import other libs
import matplotlib.pyplot as plt
import pickle
import gzip

# We will not use anything else

## Read data and explore a little bit

The specific data we will use in this lab is the famous MNIST dataset of hand-written digit images. [The original MNIST dataset](http://yann.lecun.com/exdb/mnist/) contains 2 sets: training set (60000 images) and test set (10000 images). The MNIST dataset we will use in this lab (the "mnist.pkl.gz" file) contains 3 sets: training set (50000 images), validation set (10000 images), and test set (10000 images); the training set and the validation set here were created by splitting the original training set (60000 images).  

The code below will read data from the "mnist.pkl.gz" file (you shold put the "mnist.pkl.gz" file to the same folder with this notebook file) and store data in 6 Numpy arrays:

- `train_X`, `train_Y`
- `val_X`, `val_Y`
- `test_X`, `test_Y`

In [ ]:
def read_mnist(mnist_file):
    '''
    Reads MNIST data.
    
    Parameters
    ----------
    mnist_file : string
        The name of the MNIST file (e.g., 'mnist.pkl.gz').
    
    Returns
    -------
    (train_X, train_Y, val_X, val_Y, test_X, test_Y) : tuple
        train_X : numpy array, shape (N=50000, d=784)
            Input vectors of the training set.
        train_Y : numpy array, shape (N=50000)
            Outputs of the training set.
        val_X : numpy array, shape (N=10000, d=784)
            Input vectors of the validation set.
        val_Y : numpy array, shape (N=10000)
            Outputs of the validation set.
        test_X : numpy array, shape (N=10000, d=784)
            Input vectors of the test set.
        test_Y : numpy array, shape (N=10000)
            Outputs of the test set.
    '''
    
    f = gzip.open(mnist_file, 'rb')
    train_data, val_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    
    train_X, train_Y = train_data
    val_X, val_Y = val_data
    test_X, test_Y = test_data    
    
    return train_X, train_Y, val_X, val_Y, test_X, test_Y

In [ ]:
train_X, train_Y, val_X, val_Y, test_X, test_Y = read_mnist('mnist.pkl.gz')

print('train_X.shape =', train_X.shape)
print('train_Y.shape =', train_Y.shape)
print('val_X.shape   =', val_X.shape)
print('val_Y.shape   =', val_Y.shape)
print('test_X.shape  =', test_X.shape)
print('test_Y.shape  =', test_Y.shape)

print('\ntrain_X: min = %.3f, max = %.3f' %(train_X.min(), train_X.max()))
print('train_Y: min = %d, max = %d' %(train_Y.min(), train_Y.max()))

## Build a good Neural Net from data

### Implement functions to train a Neural Net

In this lab, we will use a simple form of Neural Net: Fully-Connected Feed-Forward Neural Net. We will use sigmoid function as activation function in hidden layers *as well as in output layer* (using sigmoid instead of softmax in output layer will make our life easier). We will have $K$ neurons in output layer with $K$ is the number of classes (in our case, $K = 10$); the output of each neuron will indicate the probability a given input vector belonging to the corresponding class (note that sum of all these probabilities will not equal 1 as softmax). We can choose the class with highest probability as the predicted class for a given input vector.


In our first step, we will implement `compute_nnet_outputs` function *(3 points)*. We will use this function not only after training (to compute outputs of our trained Neural Net with new input vectors) but also during the training process (which we will tackle in our next step).

In [ ]:
def sigmoid(S):
    '''
    Computes sigmoid function for each element of array S.
    You can use this function in `compute_nnet_outputs` function.
    '''
    return 1 / (1 + np.exp(-S))

def compute_nnet_outputs(Ws, X, need_all_layer_outputs):
    '''
    Computes the outputs of Neural Net by forward propagating X through the net.
    
    Parameters
    ----------
    Ws : list of numpy arrays
        Ws[l-1] is W of layer l with l >= 1 (layer 0 is input layer, 
        it doesn't have W); W of layer l will have the shape of 
        (d^(l-1)+1, d^(l)), where  d^(l-1) is the number of neurons 
        (not count the +1 neuron) of layer l-1 and  d^(l) is the number of 
        neurons (not count the +1 neuron) of layer l.
    X : numpy array, shape (N, d+1)
        The matrix of input vectors (each row corresponds to an input vector); 
        the first column of this matrix is all ones (corresponding to x_0).
    need_all_layer_outputs : bool
        If this var is true, we'll return a list of layer's-outputs (we'll 
        need this list when training); otherwise, we'll return the final 
        layer's output.
    
    Returns
    -------
    If `need_all_layer_outputs` is false, return
        A : numpy array, shape (N, K=10)
            The maxtrix of output vectors of final layer; each row is an 
            output vector (containing each class's probability given the 
            corresponding input vector).
    Else, return
        As : list of numpy arrays
            As[l] is the matrix of output vectors of layer l (l=0 will 
            correspond to input layer); each row is an output vector 
            (corresponding to an input vector); if layer l is not the final 
            layer, the first column of this matrix is all ones.
    '''    
    
    # TODO
    # NOTE: to make the code run fast, you should use Numpy operations on
    # Numpy arrays; your code should have only one loop to loop through layers
    

In [ ]:
# CHECK THE CORRECTNESS OF YOUR COMPUTE_NNET_OUTPUTS FUNCTION

# A small X with 4 rows corresponding to 4 input vectors
X = np.array([[1.0, 0.9, 0.9], 
              [1.0, 0.5, 0.4], 
              [1.0, 0.4, 0.5],
              [1.0, 0.1, 0.7]])
# A small neural net: 
# 2 input neurons - 3 hidden neurons - 2 hidden neurons - 1 output neurons
# (not counting +1 neurons)
Ws = [np.array([[-0.3 ,  0.2 ,  0.5 ],
                [-0.1 , -0.2 , -0.35],
                [ 0.45, -0.7 , -0.7 ]]),
      np.array([[ 0.3 , -0.05],
                [ 0.6 ,  0.3 ],
                [-0.8 , -0.3 ],
                [ 0.4 , -0.45]]),
      np.array([[-0.3 ],
                [ 0.5 ],
                [-0.45]])]

# Check your compute_nnet_outputs function!
A = compute_nnet_outputs(Ws, X, False)
assert np.array_equal(np.round(A, 5),
                      np.array([[0.45109],
                                [0.45199],
                                [0.4521 ],
                                [0.45247]]))
As = compute_nnet_outputs(Ws, X, True)
assert len(As) == 4
assert np.array_equal(np.round(As[0], 5),
                      np.array([[1. , 0.9, 0.9],
                                [1. , 0.5, 0.4],
                                [1. , 0.4, 0.5],
                                [1. , 0.1, 0.7]]))
assert np.array_equal(np.round(As[1], 5),
                      np.array([[1.     , 0.50375, 0.35206, 0.39055],
                                [1.     , 0.4576 , 0.45512, 0.51125],
                                [1.     , 0.47128, 0.44275, 0.5025 ],
                                [1.     , 0.50125, 0.42311, 0.49375]]))
assert np.array_equal(np.round(As[2], 5),
                      np.array([[1.     , 0.617  , 0.45506],
                                [1.     , 0.60228, 0.43062],
                                [1.     , 0.60577, 0.4335 ],
                                [1.     , 0.61296, 0.43813]]))
assert np.array_equal(np.round(As[3], 5),
                      np.array([[0.45109],
                                [0.45199],
                                [0.4521 ],
                                [0.45247]]))

Now, we will implement our main function: `train_nnet` *(4 points)*. Basically, in this function, we will find values of our Neural Net weights by minimizing mean cross-entropy error on the training set  $\{(\textbf{x}^{(1)}, y^{(1)}), ..., (\textbf{x}^{(N)}, y^{(N)})\}$:
$$E(w\text{'s of our Net}) = \frac{1}{N} \sum_{n=1}^Ne(h(\textbf{x}^{(n)}), \textbf{y}^{(n)}))$$
where:
- $h(\textbf{x}^{(n)}) \in \mathbb{R}^{K}$ is the output vector (containing probabilities) of our Neural Net corresponding to the input vector $\textbf{x}^{(n)}$
- $\textbf{y}^{(n)} \in \mathbb{R}^{K}$ is the one-hot representation of $y^{(n)}$ (can you see the difference between  $\textbf{y}^{(n)}$ and $y^{(n)}$?) \
For example, in our case $K=10$ (10 digits):
    - If $y^{(n)}=0$ (digit 0) then $\textbf{y}^{(n)}=[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]^T$
    - If $y^{(n)}=1$ (digit 1) then $\textbf{y}^{(n)}=[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]^T$
- $e(h(\textbf{x}^{(n)}), \textbf{y}^{(n)}))$ is the cross-entropy error on a single training pair $(\textbf{x}^{(n)}, y^{(n)})$:
$$e(h(\textbf{x}^{(n)}), \textbf{y}^{(n)})) = \sum_{k=1}^K - y^{(n)}_k \ln h(\textbf{x}^{(n)})_k - (1 - y^{(n)}_k)\ln (1 - h(\textbf{x}^{(n)})_k)$$

If we use GD (Gradient Descent) to minimize $E(w\text{'s of our Net})$, we will need to compute partial derivatives of $E$ with respect to $w\text{'s}$. To compute these partial derivatives, we will compute partial derivatives of $e$ (cross-entropy error on a single training pair $\mathbf{x}$ and $y$) with respect to $w\text{'s}$, and then take average over all training pairs. But it will slow when our training set is big. SGD (Stochastic Gradient Descent) approximates the average of partial derivatives over all training pairs by the average of partial derivatives over a mini-batch (a subset of training pairs); so, it's faster than GD. Another name of SGD: "túy quyền" ;-). In `train_nnet` function below, we will use SGD.

Let's focus on how to compute the partial derivative of $e$ (cross-entropy error on a single training pair $\mathbf{x}$ and $y$) with respect to a weight $w$ between a neuron i in layer l-1 and a neuron j in layer l. After applying chain rule, we have the formula: 
$$\text{this partial derivative} = \text{output of neuron i in layer l-1} \times \text{delta of neuron j in layer l}$$
with delta of a neuron is the partial derivative of $e$ with respect to the weighted sum (value before applying activation function) of that neuron.

I have implemented most of `train_nnet` function for you. Your job is to write 3 lines of code corresponding to 3/4 tasks below:

**Task 1. Compute delta's of the last layer (on a mini-batch)**

For example, let's consider $K=3$ (3 classes) and a mini-batch with size of 2 (2 training pairs), if:

        A = np.array([[0.8, 0.7, 0.6],
                      [0.5, 0.6, 0.5]]) # Output vectors of last layer 
                                          on this mini-batch
        mb_Y = np.array([[0, 1, 0],
                         [1, 0, 0]]) # Correct one-hot output vectors 
                                       of this mini-batch
then delta's of the last layer on this mini-batch will be:

        delta = np.array([[ 0.8, -0.3,  0.6],
                          [-0.5,  0.6,  0.5]])
You should do the math to figure out the exact formula to compute delta's of the last layer. In case it's difficult for you, try your luck and guess the formula from the result above ;-). Again, when implementing this task, you should use Numpy operations on Numpy arrays; this task should be done with just one line of code.

**Task 2. Compute gradient - a collection of partial derivatives - of the last layer from delta's of the last layer and outputs of the previous layer (on a mini-batch)**

For example, let's consider $K=3$ (3 classes; it's also the number of neurons in the last layer L), the previous layer (layer L-1) with 4 neurons, and a mini-batch with size of 2 (2 training pairs), if:

        delta = np.array([[ 0.8, -0.3,  0.6],
                          [-0.5,  0.6,  0.5]]) # Delta vectors of last layer 
                                                 on this mini-batch
        A = np.array([[1.0, 0.5, 0.1, 0.3, 0.2],
                      [1.0, 0.9, 0.8, 0.7, 0.1]]) # Output vectors of the previous layer 
                                                    on this mini-batch
then the gradient of the last layer will be (it has the same shape with `W` array of the last layer):

        grad = np.array([[ 0.15 ,  0.15 ,  0.55 ],
                         [-0.025,  0.195,  0.375],
                         [-0.16 ,  0.225,  0.23 ],
                         [-0.055,  0.165,  0.265],
                         [ 0.055,  0.   ,  0.085]]) # For clarity, I use 
                                                      np.round(..., 3) to show this
How is `grad` computed from `delta` and `A`? Hint: I have mentioned above how to compute the partial derivative of cross-entropy error on a single training pair with respect to a weight; for a mini-batch, just compute it for each training pair and then take average.

Again, when implementing this task, you should use Numpy operations on Numpy arrays; this task should be done with just one line of code.

**Task 3. Compute delta's of layer l from delta's of layer l+1 (on a mini-batch)**

(Note that we will not compute delta's of +1 neurons)

This is the most difficult task, but I have done it for you :-). 


**Task 4. Compute gradient of layer l from delta of layer l and outputs of layer l-1 (on a mini-batch)**

It's similar to task 2.

In [ ]:
def train_nnet(X, Y, val_X, val_Y, 
               hid_layer_sizes, 
               mb_size, learning_rate, max_epoch):
    '''
    Trains Neural Net on the dataset (X, Y); also prints out mean binary error 
    (the percentage of misclassified data points) on training set and 
    validation set during training
    Cost function: mean cross-entropy error
    Optimization algorithm: SGD
    
    Parameters
    ----------
    X : numpy array, shape (N, d + 1)
        The matrix of input vectors (each row corresponds to an input vector); 
        the first column of this matrix is all ones (corresponding to x_0).
    Y : numpy array, shape (N,) 
        The vector of outputs.
    val_X, val_Y : validation data, similar to X and Y
    hid_layer_sizes : list
        The list of hidden layer sizes; e.g., hid_layer_sizes = [20, 10] means
        the Net has 2 hidden layers, the 1st one has 20 neurons, and the 2nd 
        one has 10 neurons (not count the +1 neurons).
    mb_size : int
        Minibatch size of SGD.
    learning_rate : float
        Learning rate of SGD.
    max_epoch : int
        After this number of epochs, we'll terminate SGD.

    Returns
    -------
    (Ws, costs, errs) : tuple
        Ws : list of numpy arrays
            Ws[l-1] is W of layer l with l >= 1 (layer 0 is input layer, 
            it doesn't have W); W of layer l will have the shape of 
            (d^(l-1)+1, d^(l)), where d^(l-1) is the number of neurons 
            (not count the +1 neuron) of layer l-1 and d^(l) is the number of 
            neurons (not count the +1 neuron) of layer l.
        costs : list, len = max_epoch
            The list of costs after each epoch.
        errs : list, len = max_epoch
            The list of mean binary errors (on the training set) after each epoch.
    '''
    
    # Prepare for training
    K = len(np.unique(Y)) # Num classes
    layer_sizes = [X.shape[1] - 1] + hid_layer_sizes + [K]
    np.random.seed(0) # This will fix the randomization; 
                      # so, you and me will have the same results
    Ws = [np.random.randn(layer_sizes[i] + 1, layer_sizes[i + 1]) 
          / np.sqrt(layer_sizes[i] + 1) 
          for i in range(len(layer_sizes) - 1)] # Secret formula to init Ws ;-)
    one_hot_Y = np.zeros((len(Y), K))
    one_hot_Y[np.arange(len(Y)), Y] = 1
    errs = [] # To save mean binary errors on training set during training
    val_errs = [] # To save mean binary errors on validation set during training
    N = len(X) # Num training examples
    rnd_idxs = np.arange(N) # Random indexes    
    
    # Train
    for epoch in range(max_epoch):
        np.random.shuffle(rnd_idxs)
        for start_idx in range(0, N, mb_size):
            # Get minibach
            mb_X = X[rnd_idxs[start_idx:start_idx+mb_size]]
            mb_Y = one_hot_Y[rnd_idxs[start_idx:start_idx+mb_size]]
            
            # Forward-prop
            As = compute_nnet_outputs(Ws, mb_X, True)
            
            # Back-prop; on the way, compute each layer's gradient and update its W
            # TODO: delta = ... (task 1)
            
            # TODO: grad = ... (task 2)
            
            Ws[-1] -= learning_rate * grad
            for i in range(2, len(Ws) + 1):
                delta = delta.dot(Ws[-i + 1].T[:, 1:]) * As[-i][:, 1:] * (1 - As[-i][:, 1:])
                # TODO: grad = ... (task 4)
                
                Ws[-i] -= learning_rate * grad
        
        # Compute training info, save it, and print it
        A = compute_nnet_outputs(Ws, X, False)
        err = np.mean(np.argmax(A, axis=1) != Y) * 100
        errs.append(err)
        val_A = compute_nnet_outputs(Ws, val_X, False)
        val_err = np.mean(np.argmax(val_A, axis=1) != val_Y) * 100
        val_errs.append(val_err)
        print('Epoch %d, train err %.3f%%, val err %.3f%%' %(epoch, err, val_err))
            
    return Ws, errs, val_errs

In [ ]:
# CHECK THE CORRECTNESS YOUR TRAIN_NNET FUCTION
Ws, train_errs, val_errs = train_nnet(train_X, train_Y, val_X, val_Y,
                                      hid_layer_sizes=[20], 
                                      mb_size=32, learning_rate=0.1, max_epoch=3)
assert np.array_equal(np.round(train_errs, 3),
                      np.array([10.982, 9.044, 7.962]))
assert np.array_equal(np.round(val_errs, 3),
                      np.array([9.82, 8.23, 7.49]))

### Use implemented functions to train different Neural Nets and choose the best

In [ ]:
# Shallow Neural Net
Ws_50, train_errs_50, val_errs_50 = \
    train_nnet(train_X, train_Y, val_X, val_Y, 
               hid_layer_sizes=[50], 
               mb_size=32, learning_rate=0.1, max_epoch=30)

In [ ]:
# Deeper Neural Net
Ws_50_50, train_errs_50_50, val_errs_50_50 = \
    train_nnet(train_X, train_Y, val_X, val_Y, 
               hid_layer_sizes=[50, 50], 
               mb_size=32, learning_rate=0.1, max_epoch=30)

In [ ]:
# Even more deeper Neural Net!
Ws_50_50_50, train_errs_50_50_50, val_errs_50_50_50 = \
    train_nnet(train_X, train_Y, val_X, val_Y, 
               hid_layer_sizes=[50, 50, 50], 
               mb_size=32, learning_rate=0.1, max_epoch=30)

Visualize results:

In [ ]:
plt.plot(np.log(train_errs_50), color='blue', 
         label='Net 50 - train err')
plt.plot(np.log(val_errs_50), color='blue', linestyle='--', 
         label='Net 50 - val err')
plt.plot(np.log(train_errs_50_50), color='green',
         label='Net 50_50 - train err')
plt.plot(np.log(val_errs_50_50), color='green', linestyle='--',
         label='Net 50_50 - val err')
plt.plot(np.log(train_errs_50_50_50), color='red',
         label='Net 50_50_50 - train err')
plt.plot(np.log(val_errs_50_50_50), color='red', linestyle='--',
         label='Net 50_50_50 - val err')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Ln of error (for clarity)');

TODO: comment on the graph above (Why is it like this? In case you don't know why, just simply say so) *(2 points)*.

We will choose the Neural Net model having smallest mean binary error on the validation set as our best Neural Net model.

## How good our Neural Net actually is

Let's compute mean binary error ($\in [0, 100]$) of our best Neural Net model on the test set and store result in `test_err` variable *(1 points)*. 

In [ ]:
# TODO


In [ ]:
# CHECK CORRECTNESS
assert np.round(test_err, 3) == 2.860